In [1]:
# settings
settings_remove_missing = True
settings_output_filename = "preprocessed.csv"
settings_output_filename_test = "preprocessed_test_public.csv"

In [2]:
import numpy as np
import pandas as pd
from datetime import datetime
import pytz


In [3]:
df_tr = pd.read_csv("train.csv")

In [4]:
df_test_public = pd.read_csv("test_public.csv")

In [5]:
if settings_remove_missing:
    df_tr = df_tr[~df_tr['MISSING_DATA']]
    # also remove CALL_TYPE B data that is missing ORIGIN_STAND?

In [15]:
def polyline_to_trip_duration(polyline):
  return max(polyline.count("[") - 2, 0) * 15

def parse_time(x):
    zone = pytz.timezone("Europe/Lisbon")
    dt = datetime.fromtimestamp(x["TIMESTAMP"], tz=zone)
    return dt.year, dt.month, dt.day, dt.hour, dt.weekday()

def parse_timestamp(x):
    zone = pytz.timezone("Europe/Lisbon")
    dt = datetime.fromtimestamp(x, tz=zone)
    return dt.year, dt.month, dt.day, dt.hour, dt.weekday()

In [7]:
# Add LEN column
df_tr["LEN"] = df_tr["POLYLINE"].apply(polyline_to_trip_duration)

In [8]:
# Add datetime columns
df_tr[["YR", "MON", "DAY", "HR", "WK"]] = df_tr[["TIMESTAMP"]].apply(parse_time, axis=1, result_type="expand")
df_test_public[["YR", "MON", "DAY", "HR", "WK"]] = df_test_public[["TIMESTAMP"]].apply(parse_time, axis=1, result_type="expand")

In [18]:
def is_holiday(timestamp):
    yr, mon, day, _hr, _wk = parse_timestamp(timestamp)
#     print(f"year {yr}, month {mon}, day {day}, hour {_hr}, week {_wk}")
    holidays = [
        (2013, 1, 1),
        (2014, 1, 1),
        (2013, 2, 12),
        (2014, 3, 4),
        (2013, 3, 29),
        (2014, 4, 18),
        (2013, 3, 31),
        (2014, 4, 20),
        (2013, 4, 25),
        (2014, 4, 25),
        (2013, 5, 1),
        (2014, 5, 1),
        (2013, 6, 10),
        (2014, 6, 10),
        (2013, 8, 15),
        (2014, 8, 15),
        (2013, 10, 5),
        (2014, 10, 5),
        (2013, 11, 1),
        (2014, 11, 1),
        (2013, 12, 1),
        (2014, 12, 1),
        (2013, 12, 25),
        (2014, 12, 25)
    ]
    if (yr, mon, day) in holidays:
        return True
    return False

def is_holidays_eve(timestamp):
    tomorrow = timestamp + (60 * 60 * 24)
    return is_holiday(tomorrow)

In [19]:
df_tr["DAY_B"] = df_tr.apply(lambda row:
        1 if is_holiday(row["TIMESTAMP"]) else 0, axis=1)
df_tr["DAY_C"] = df_tr.apply(lambda row:
        1 if is_holidays_eve(row["TIMESTAMP"]) else 0, axis=1)

In [33]:
df_test_public["DAY_B"] = df_test_public.apply(lambda row:
        1 if is_holiday(row["TIMESTAMP"]) else 0, axis=1)
df_test_public["DAY_C"] = df_test_public.apply(lambda row:
        1 if is_holidays_eve(row["TIMESTAMP"]) else 0, axis=1)

In [34]:
print(df_tr.columns)

Index(['TRIP_ID', 'CALL_TYPE', 'ORIGIN_CALL', 'ORIGIN_STAND', 'TAXI_ID',
       'TIMESTAMP', 'DAY_TYPE', 'MISSING_DATA', 'POLYLINE', 'LEN', 'YR', 'MON',
       'DAY', 'HR', 'WK', 'DAY_B', 'DAY_C', 'ORDERED_STAND', 'ORDERED_CALL',
       'ORDERED_TAXI_ID', 'NORM_TIMESTAMP', 'ORDERED_WK', 'ORDERED_DAY',
       'ORDERED_MON', 'TYPE_A', 'TYPE_B', 'TYPE_C', 'START_LOC', 'START_LAT',
       'START_LON'],
      dtype='object')


In [35]:
df_tr = df_tr.dropna(subset=["LEN"])

In [36]:
def make_sorted_column(original_column, ordered_column):
    values = df_tr[original_column].unique()
    value_lens = [None] * len(values)

    mean_value = 0
    for i, value in enumerate(values):
        mean_len = df_tr[df_tr[original_column] == value]["LEN"].mean()
        if (np.isnan(value)):
            mean_len = df_tr[np.isnan(df_tr[original_column])]["LEN"].mean()
        value_lens[i] = [value, mean_len]
        mean_value += mean_len

    mean_value /= len(values)
    sorted_lens = sorted(value_lens, key=lambda x: x[1])
    
    value_to_sorted = {}
#     value_to_sorted[np.float64('nan')] = mean_value
    for i, value in enumerate(sorted_lens):
        value_to_sorted[value[0]] = i
    
#     public_values = df_test_public[original_column].unique()
#     for value in public_values:
#         print(value)
    
    df_tr[ordered_column] = df_tr[original_column].map(value_to_sorted)
    df_test_public[ordered_column] = df_test_public[original_column].map(value_to_sorted)
    
    df_tr[ordered_column] = df_tr[ordered_column].fillna(mean_value)
    df_test_public[ordered_column] = df_test_public[ordered_column].fillna(mean_value)


In [37]:
make_sorted_column("ORIGIN_STAND", "ORDERED_STAND")

In [23]:
make_sorted_column("ORIGIN_CALL", "ORDERED_CALL")

In [24]:
make_sorted_column("TAXI_ID", "ORDERED_TAXI_ID")

In [25]:
df_tr["NORM_TIMESTAMP"] = df_tr["TIMESTAMP"]
df_test_public["NORM_TIMESTAMP"] = df_test_public["TIMESTAMP"]

In [26]:
make_sorted_column("WK", "ORDERED_WK")
make_sorted_column("DAY", "ORDERED_DAY")
make_sorted_column("MON", "ORDERED_MON")

In [27]:
def encode_call_type(df):
    df['TYPE_A'] = np.where(df['CALL_TYPE'] == 'A', 1, 0)
    df['TYPE_B'] = np.where(df['CALL_TYPE'] == 'B', 1, 0)
    df['TYPE_C'] = np.where(df['CALL_TYPE'] == 'C', 1, 0)

encode_call_type(df_tr)
encode_call_type(df_test_public)

In [28]:
df_stand_loc = pd.read_csv("metaData_taxistandsID_name_GPSlocation.csv")

In [29]:
calls = df_stand_loc["ID"].unique()

call_lens = [None] * len(calls)

call_id_to_loc = {}
call_id_to_lat = {}
call_id_to_lon = {}
for i, call in enumerate(calls):
    call_id_to_loc[call] = df_stand_loc.loc[i, "Descricao"]
    call_id_to_lat[call] = df_stand_loc.loc[i, "Latitude"]
    call_id_to_lon[call] = df_stand_loc.loc[i, "Longitude"]

df_tr["START_LOC"] = df_tr["ORIGIN_STAND"].map(call_id_to_loc)
df_tr["START_LAT"] = df_tr["ORIGIN_STAND"].map(call_id_to_lat)
df_tr["START_LON"] = df_tr["ORIGIN_STAND"].map(call_id_to_lon)
df_test_public["START_LOC"] = df_test_public["ORIGIN_STAND"].map(call_id_to_loc)
df_test_public["START_LAT"] = df_test_public["ORIGIN_STAND"].map(call_id_to_lat)
df_test_public["START_LON"] = df_test_public["ORIGIN_STAND"].map(call_id_to_lon)

In [30]:
df_tr.head()

,TRIP_ID,CALL_TYPE,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,TIMESTAMP,DAY_TYPE,MISSING_DATA,POLYLINE,LEN,...,NORM_TIMESTAMP,ORDERED_WK,ORDERED_DAY,ORDERED_MON,TYPE_A,TYPE_B,TYPE_C,START_LOC,START_LAT,START_LON
0,1372636858620000589,C,NaN,NaN,20000589,1372636858,A,False,"[[-8.618643,41.141412],[-8.618499,41.141376],[...",330,...,1372636858,5,0,1,0,0,1,NaN,NaN,NaN
1,1372637303620000596,B,NaN,7.0,20000596,1372637303,A,False,"[[-8.639847,41.159826],[-8.640351,41.159871],[...",270,...,1372637303,5,0,1,0,1,0,Av. Boavista,41.15998,-8.641984
2,1372636951620000320,C,NaN,NaN,20000320,1372636951,A,False,"[[-8.612964,41.140359],[-8.613378,41.14035],[-...",960,...,1372636951,5,0,1,0,0,1,NaN,NaN,NaN
3,1372636854620000520,C,NaN,NaN,20000520,1372636854,A,False,"[[-8.574678,41.151951],[-8.574705,41.151942],[...",630,...,1372636854,5,0,1,0,0,1,NaN,NaN,NaN
4,1372637091620000337,C,NaN,NaN,20000337,1372637091,A,False,"[[-8.645994,41.18049],[-8.645949,41.180517],[-...",420,...,1372637091,5,0,1,0,0,1,NaN,NaN,NaN


In [38]:
# normalization
from sklearn.preprocessing import StandardScaler

feature_cols = ["MON", "DAY", "HR", "WK", "ORDERED_STAND", "ORDERED_CALL", "ORDERED_TAXI_ID"
            ,"TYPE_A", "TYPE_B", "TYPE_C",
            "ORDERED_WK", "ORDERED_DAY", "ORDERED_MON", "DAY_B", "DAY_C", "NORM_TIMESTAMP",
            "START_LAT", "START_LON"]

scaler = StandardScaler()

scaler.fit(df_tr[feature_cols])

df_tr[feature_cols] = scaler.transform(df_tr[feature_cols])

df_test_public[feature_cols] = scaler.transform(df_test_public[feature_cols])



In [39]:
df_tr.to_csv(settings_output_filename, index=False)

In [40]:
df_test_public.to_csv(settings_output_filename_test, index=False)